# IBM DS Capstone Project - Week 3 Part 2

Import required modules

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib

Create a new dataframe with the required columns:

In [3]:
cols = ['PostalCode', 'Borough','Neighbourhood']
df = pd.DataFrame(columns=cols)

Retrieve the web page and build a new soap object

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = urllib.request.urlopen(url)
content = r.read()
soup = BeautifulSoup(content)

Go over all the rows in the table and add the data corresponding to each table row to a new dictionary:

In [5]:
data = {}
rows = soup.tbody.findAll('tr')
for r in rows:
    cols = r.findAll('td')
    if len(cols) < 3:
        continue
    postal_code = cols[0].text.strip()
    borough = cols[1].text.strip()
    neighbourhood = cols[2].text.strip()
    
    # ignore not assigned boroughs
    if (borough == 'Not assigned'):
        continue
    
    # update not assigned neighbourhoods
    if (neighbourhood == 'Not assigned'):
        neighbourhood = borough
    
    if (postal_code and borough and neighbourhood):
        if postal_code in data:
            new_n = data[postal_code]['Neighbourhood'] + ',' + neighbourhood
            data[postal_code] = {'PostalCode': postal_code, 'Borough': borough, 'Neighbourhood': new_n}
        else:
            data[postal_code] = {'PostalCode': postal_code, 'Borough': borough, 'Neighbourhood': neighbourhood}

For each entry in the dictionary add a row to the new dataframe.

In [6]:
for k, v in data.items():
    df = df.append(v, ignore_index=True)

Inspect the new dataframe:

In [7]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## Part 2

Retrieve geospatinal data from the web, and created a dictionary with information about each postal code:

In [9]:
import csv

url = 'https://cocl.us/Geospatial_data'
r = urllib.request.urlopen(url)
data = r.read().decode('utf-8')
read = csv.reader(data.splitlines()[1:])

geo = {}
for row in read:
    geo[row[0]] = row

Define two function that given the postal code, retrieve the latitude and longitude from the created dictionary:

In [11]:
def get_lat(postal_code):
    return(geo[postal_code][1])
def get_lon(postal_code):
    return(geo[postal_code][2])

Add the Latitude and Longitude columns to the dataframe:

In [25]:
df['Latitude'] = df.apply(lambda row: get_lat(row['PostalCode']), axis=1)
df['Longitude'] = df.apply(lambda row: get_lon(row['PostalCode']), axis=1)

Inspect the new columns in the dataframe

In [26]:
df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
